In [1]:
#Read before running
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

**Preliminary exploratory data analysis:**
* Demonstrate that the dataset can be read from the web into R 
* Clean and wrangle your data into a tidy format

In [9]:
#read red wine data, assign wine-type label and new column names
red_wine_data <- read_delim("winequality-red (1).csv", delim = ";")%>%
                    mutate(quality = as.factor(quality))%>%
                    mutate(wine_type = as.factor("red"))%>%
                    setNames (c("fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar", "chlorides", "free_sulfur_dioxide", "total_sulfur_dioxide", "density", "pH", "sulphates", "alcohol", "quality", "wine_type"))

#read white wine data, assign wine-type label and new column names
white_wine_data <- read_delim("winequality-white.csv", delim = ";")%>%
                    mutate(quality = as.factor(quality))%>%
                    mutate(wine_type = as.factor("white"))%>%
                    setNames (c("fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar", "chlorides", "free_sulfur_dioxide", "total_sulfur_dioxide", "density", "pH", "sulphates", "alcohol", "quality", "wine_type"))
#combine both wine types into one dataframe
wine_data <- rbind(red_wine_data, white_wine_data)


Parsed with column specification:
cols(
  `fixed acidity` = col_double(),
  `volatile acidity` = col_double(),
  `citric acid` = col_double(),
  `residual sugar` = col_double(),
  chlorides = col_double(),
  `free sulfur dioxide` = col_double(),
  `total sulfur dioxide` = col_double(),
  density = col_double(),
  pH = col_double(),
  sulphates = col_double(),
  alcohol = col_double(),
  quality = col_double()
)

Parsed with column specification:
cols(
  `fixed acidity` = col_double(),
  `volatile acidity` = col_double(),
  `citric acid` = col_double(),
  `residual sugar` = col_double(),
  chlorides = col_double(),
  `free sulfur dioxide` = col_double(),
  `total sulfur dioxide` = col_double(),
  density = col_double(),
  pH = col_double(),
  sulphates = col_double(),
  alcohol = col_double(),
  quality = col_double()
)



* Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
* Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [4]:
#split dataset into training and testing data
wine_split <- initial_split(wine_data, prop = 0.75, strata = quality)
wine_training <- training(wine_split)
wine_testing <- testing(wine_split)

In [5]:
glimpse(wine_training)

Rows: 4,874
Columns: 13
$ fixed_acidity        <dbl> 7.8, 7.8, 11.2, 7.4, 7.9, 7.3, 6.7, 7.5, 5.6, 8.…
$ volatile_acidity     <dbl> 0.880, 0.760, 0.280, 0.700, 0.600, 0.650, 0.580,…
$ citric_acid          <dbl> 0.00, 0.04, 0.56, 0.00, 0.06, 0.00, 0.08, 0.36, …
$ residual_sugar       <dbl> 2.6, 2.3, 1.9, 1.9, 1.6, 1.2, 1.8, 6.1, 1.6, 3.9…
$ chlorides            <dbl> 0.098, 0.092, 0.075, 0.076, 0.069, 0.065, 0.097,…
$ free_sulfur_dioxide  <dbl> 25, 15, 17, 11, 15, 15, 15, 17, 16, 51, 35, 16, …
$ total_sulfur_dioxide <dbl> 67, 54, 60, 34, 59, 21, 65, 102, 59, 148, 103, 5…
$ density              <dbl> 0.9968, 0.9970, 0.9980, 0.9978, 0.9964, 0.9946, …
$ pH                   <dbl> 3.20, 3.26, 3.16, 3.51, 3.30, 3.39, 3.28, 3.35, …
$ sulphates            <dbl> 0.68, 0.65, 0.58, 0.56, 0.46, 0.47, 0.54, 0.80, …
$ alcohol              <dbl> 9.8, 9.8, 9.8, 9.4, 9.4, 10.0, 9.2, 10.5, 9.9, 9…
$ quality              <fct> 5, 5, 6, 5, 5, 7, 5, 5, 5, 5, 7, 5, 4, 6, 6, 5, …
$ wine_type            <fct>

Extracting all the categories that may be predicted:

In [6]:
classes <- wine_training %>% pull(quality) %>% levels()
classes

[1] "3" "4" "5" "6" "7" "8" "9"

Find the number and percentage of the categories in the dataset

In [7]:
num_obs <- nrow(wine_training)

wine_training %>%
  group_by(quality) %>%
  summarize(
    count = n(),
    percentage = n() / num_obs * 100
  )

`summarise()` ungrouping output (override with `.groups` argument)



quality,count,percentage
<fct>,<int>,<dbl>
3,23,0.47189167
4,168,3.44686089
5,1598,32.78621256
6,2131,43.72178908
7,810,16.61879360
8,142,2.91341814
9,2,0.04103406


The predictor variables that will be used:


* Fixed acidity (g(tartaric acid)/dm3)
* Volatile acidity (g(acetic acid)/dm3)
* Residual sugar (g/dm3)
* Free sulfur dioxide (mg/dm3)
* Total sulfur dioxide (mg/dm3)
* Alcohol (%vol)


In [13]:
predictor_means <- wine_training %>%
    select(fixed_acidity, volatile_acidity, residual_sugar, free_sulfur_dioxide, total_sulfur_dioxide, alcohol)%>%
    colMeans()
    
predictor_means

fixed_acidity     volatile_acidity       residual_sugar 
           7.2192142            0.3390449            5.4590890 
 free_sulfur_dioxide total_sulfur_dioxide              alcohol 
          30.4394748          115.7232253           10.4896088